# Reproducing error for PeriodIndex and ipywidget

In [1]:
#!pip install tabula-py cbsodata

In [2]:
#from IPython.display import Markdown as md
#import numpy as np
import pandas as pd
import tabula
from datetime import datetime
#import json
import cbsodata as cbs

## Collecting data

### CBS
https://opendata.cbs.nl/statline/portal.html

In [3]:
print('Importing: Bevolkingsontwikkeling; maand en jaar 2002-now (Population statistics, month and year)')
population = pd.DataFrame(cbs.get_data('37230ned'))
print('DONE')

Importing: Bevolkingsontwikkeling; maand en jaar 2002-now (Population statistics, month and year)
DONE


In [4]:
print('Preparing Dataframe for analyses')
print('Dropping unnecessary columns')
population = population.drop(columns=['ID', 'BevolkingAanHetBeginVanDePeriode_1','TotaleVestiging_4',
       'VestigingVanuitEenAndereGemeente_5', 'Immigratie_6',
       'TotaalVertrekInclAdmCorrecties_7', 'VertrekNaarAndereGemeente_8',
       'EmigratieInclusiefAdmCorrecties_9', 'OverigeCorrecties_10',
       'Bevolkingsgroei_11', 'BevolkingsgroeiRelatief_12',
       'BevolkingsgroeiSinds1Januari_13',
       'BevolkingsgroeiSinds1JanuariRela_14'])
population = population.rename(columns={"LevendGeborenKinderen_2": "Born", "Overledenen_3": "Deceased", "BevolkingAanHetEindeVanDePeriode_15": "Population"})
print('DONE')

Preparing Dataframe for analyses
Dropping unnecessary columns
DONE


In [5]:
print('Setting Period')
population['year'] = population.loc[:,'Perioden'].str.slice(stop=4)
population['month'] = population.loc[:,'Perioden'].str.slice(start=5)
population = population.loc[population['month'] != '',:]
population['month'] = population.loc[:,'month'].replace({
    'januari': '01',
    'februari': '02',
    'maart': '03',
    'april': '04',
    'mei': '05',
    'juni': '06',
    'juli': '07',
    'augustus': '08',
    'september': '09',
    'oktober': '10',
    'november': '11',
    'december': '12'})

Setting Period


In [6]:
population

,RegioS,Perioden,Born,Deceased,Population,year,month
0,Nederland,2002 januari,17019.0,13469.0,16112900.0,2002,01
1,Nederland,2002 februari,15448.0,11735.0,16118953.0,2002,02
2,Nederland,2002 maart,16792.0,13281.0,16124125.0,2002,03
3,Nederland,2002 april,15995.0,11968.0,16130246.0,2002,04
4,Nederland,2002 mei,16800.0,11623.0,16136807.0,2002,05
...,...,...,...,...,...,...,...
147739,Zwolle,2020 mei,139.0,59.0,129152.0,2020,05
147740,Zwolle,2020 juni,97.0,72.0,129142.0,2020,06
147741,Zwolle,2020 juli,131.0,61.0,129213.0,2020,07
147742,Zwolle,2020 augustus,120.0,74.0,129352.0,2020,08


## so far so good

In [7]:
population = population.set_index(population['year'] + '-' + population['month'])
population.index = pd.PeriodIndex(population.index, freq='M')

# now the commands to reproduce the error

In [8]:
population.index

PeriodIndex(['2002-01', '2002-02', '2002-03', '2002-04', '2002-05', '2002-06',
             '2002-07', '2002-08', '2002-09', '2002-10',
             ...
             '2019-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05',
             '2020-06', '2020-07', '2020-08', '2020-09'],
            dtype='period[M]', length=136800, freq='M')

In [9]:
population

ValueError: Can't clean for JSON: Period('2002-01', 'M')

In [10]:
print(population)

            RegioS        Perioden     Born  Deceased  Population  year month
2002-01  Nederland    2002 januari  17019.0   13469.0  16112900.0  2002    01
2002-02  Nederland   2002 februari  15448.0   11735.0  16118953.0  2002    02
2002-03  Nederland      2002 maart  16792.0   13281.0  16124125.0  2002    03
2002-04  Nederland      2002 april  15995.0   11968.0  16130246.0  2002    04
2002-05  Nederland        2002 mei  16800.0   11623.0  16136807.0  2002    05
...            ...             ...      ...       ...         ...   ...   ...
2020-05     Zwolle        2020 mei    139.0      59.0    129152.0  2020    05
2020-06     Zwolle       2020 juni     97.0      72.0    129142.0  2020    06
2020-07     Zwolle       2020 juli    131.0      61.0    129213.0  2020    07
2020-08     Zwolle   2020 augustus    120.0      74.0    129352.0  2020    08
2020-09     Zwolle  2020 september    130.0      81.0    129465.0  2020    09

[136800 rows x 7 columns]


In [11]:
population['RegioS']

2002-01    Nederland
2002-02    Nederland
2002-03    Nederland
2002-04    Nederland
2002-05    Nederland
             ...    
2020-05       Zwolle
2020-06       Zwolle
2020-07       Zwolle
2020-08       Zwolle
2020-09       Zwolle
Freq: M, Name: RegioS, Length: 136800, dtype: object

In [12]:
population[['RegioS','Perioden']]

ValueError: Can't clean for JSON: Period('2002-01', 'M')

In [13]:
print(population[['RegioS','Perioden']])

            RegioS        Perioden
2002-01  Nederland    2002 januari
2002-02  Nederland   2002 februari
2002-03  Nederland      2002 maart
2002-04  Nederland      2002 april
2002-05  Nederland        2002 mei
...            ...             ...
2020-05     Zwolle        2020 mei
2020-06     Zwolle       2020 juni
2020-07     Zwolle       2020 juli
2020-08     Zwolle   2020 augustus
2020-09     Zwolle  2020 september

[136800 rows x 2 columns]


In [19]:
!uname -a

Linux p-6aa43c31-e561-4258-b958-6792614774f6 4.14.198-152.320.amzn2.x86_64 #1 SMP Wed Sep 23 23:57:28 UTC 2020 x86_64 GNU/Linux


In [20]:
!jupyter --version

jupyter core     : 4.6.3
jupyter-notebook : 7.0.0.dev0
qtconsole        : not installed
ipython          : 7.18.1
ipykernel        : 5.3.4
jupyter client   : 6.1.7
jupyter lab      : not installed
nbconvert        : 6.0.7
ipywidgets       : 7.5.1
nbformat         : 5.0.7
traitlets        : 5.0.4


In [21]:
!jupyter troubleshoot

$PATH:
	/opt/venv/bin
	/usr/local/bin
	/usr/local/sbin
	/usr/local/bin
	/usr/sbin
	/usr/bin
	/sbin
	/bin

sys.path:
	/opt/venv/bin
	/usr/local/lib/python37.zip
	/usr/local/lib/python3.7
	/usr/local/lib/python3.7/lib-dynload
	/opt/venv/lib/python3.7/site-packages
	/opt/venv/lib/python3.7/site-packages/python_language_server-deepnote_0.03-py3.7.egg
	/opt/venv/lib/python3.7/site-packages/ujson-4.0.1-py3.7-linux-x86_64.egg
	/opt/venv/lib/python3.7/site-packages/pluggy-1.0.0.dev0-py3.7.egg
	/opt/venv/lib/python3.7/site-packages/python_jsonrpc_server-0.4.0-py3.7.egg

sys.executable:
	/opt/venv/bin/python

sys.version:
	3.7.9 (default, Sep 10 2020, 17:21:15) 
	[GCC 8.3.0]

platform.platform():
	Linux-4.14.198-152.320.amzn2.x86_64-x86_64-with-debian-10.6

which -a jupyter:
	/opt/venv/bin/jupyter

pip list:
	Package                Version
	---------------------- -------------
	async-generator        1.10
	attrs                  20.2.0
	backcall               0.2.0
	bleach                 3.2.1


In [26]:
!jupyter lab --debug

Traceback (most recent call last):
  File "/opt/venv/bin/jupyter", line 8, in <module>
    sys.exit(main())
  File "/opt/venv/lib/python3.7/site-packages/jupyter_core/command.py", line 247, in main
    command = _jupyter_abspath(subcommand)
  File "/opt/venv/lib/python3.7/site-packages/jupyter_core/command.py", line 134, in _jupyter_abspath
    'Jupyter command `{}` not found.'.format(jupyter_subcommand)
Exception: Jupyter command `jupyter-lab` not found.
